# 自定义层

使用的主要数据结构是Layer


实现自定义层的最佳方法是扩展tf.keras.layers.Layer类并实现。
* __init__: 可以在其中进行所有与输入无关的初始化，定义相关的层
* build: 知道输入张量的形状并可以进行其余的初始化
* call: 在这里进行前向传播


注意：不一定需要在build中创建变量，也可以在__init__中创建它们。

tf.keras.Model和tf.keras.layers.Layer有什么区别和联系？
* 通过继承 tf.keras.Model 编写自己的模型类
* 通过继承 tf.keras.layers.Layer 编写自己的层
* tf.keras中的模型和层都是继承tf.Module实现的
* tf.keras.Model继承tf.keras.layers.Layer实现的

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [4]:
from sklearn import datasets
iris = datasets.load_iris()

In [5]:
data = iris.data
target = iris.target

In [6]:
data.shape

(150, 4)

In [7]:
target.shape

(150,)

## 方法一 baseline

In [9]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=1, input_dim=4):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units), dtype='float32'), trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,), dtype='float32'), trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
x = tf.constant(data)
linear_layer = Linear(units=1, input_dim=4)
y = linear_layer(x)
print(y.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


## 方法二 使用self.add_weight创建变量

In [10]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=1, input_dim=4):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(units,), initializer='zeros', trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
x = tf.constant(data)
linear_layer = Linear(units=1, input_dim=4)
y = linear_layer(x)
print(y.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


## 方法三：build函数中创建变量

In [15]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='random_normal', trainable=True)
        
        super(Linear, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
x = tf.constant(data)
linear_layer = Linear(units = 1)
y = linear_layer(x)
print(y.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


## 添加不可训练的参数

In [16]:
class Linear(tf.keras.layers.Layer):
    
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='random_normal', trainable=False)
        super(Linear, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
x = tf.constant(data)
linear_layer = Linear(units = 1)
y = linear_layer(x)
print(y.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


In [17]:
print('weight:', linear_layer.weights)
print('non-trainable weight:', linear_layer.non_trainable_weights)
print('trainable weight:', linear_layer.trainable_weights)

weight: [<tf.Variable 'linear_6/Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.01367783],
       [-0.01497129],
       [ 0.0109102 ],
       [-0.04303289]], dtype=float32)>, <tf.Variable 'linear_6/Variable:0' shape=(1,) dtype=float32, numpy=array([0.04851456], dtype=float32)>]
non-trainable weight: [<tf.Variable 'linear_6/Variable:0' shape=(1,) dtype=float32, numpy=array([0.04851456], dtype=float32)>]
trainable weight: [<tf.Variable 'linear_6/Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.01367783],
       [-0.01497129],
       [ 0.0109102 ],
       [-0.04303289]], dtype=float32)>]


### 注意一：如果要让自定义的Layer通过functional API组合成模型时可以序列化，需要自定义get_config方法

In [103]:
import tensorflow as tf
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        self.units = units
        super(MyDense, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True, name='w')
        self.b = self.add_weight(shape=(self.units,), initializer='random_normal', trainable=True, name='b')
        super(MyDense, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
#     def get_config(self):
#         config = super(MyDense, self).get_config()
#         config.update({'units': self.units})
#         return config

In [104]:
from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
labels = iris.target

In [105]:
inputs = tf.keras.Input(shape=(4,))
x = MyDense(units=16)(inputs)
x = tf.nn.tanh(x)
x = MyDense(units=3)(x)
predictions = tf.nn.softmax(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

In [106]:
import numpy as np

In [107]:
data = np.concatenate((data, labels.reshape(150,1)), axis=-1)
np.random.shuffle(data)

In [108]:
labels = data[:, -1]
data = data[:, :4]

In [109]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(data, labels, batch_size=32, epochs=10, shuffle=True)

Epoch 1/10
5/5 [==============================] - 0s 1ms/step - loss: 1.1049 - sparse_categorical_accuracy: 0.3333
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 1.1021 - sparse_categorical_accuracy: 0.3333
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 1.0990 - sparse_categorical_accuracy: 0.3333
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: 1.0964 - sparse_categorical_accuracy: 0.3800
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 1.0937 - sparse_categorical_accuracy: 0.6333
Epoch 6/10
5/5 [==============================] - 0s 2ms/step - loss: 1.0914 - sparse_categorical_accuracy: 0.6667
Epoch 7/10
5/5 [==============================] - 0s 2ms/step - loss: 1.0886 - sparse_categorical_accuracy: 0.6667
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 1.0857 - sparse_categorical_accuracy: 0.6667
Epoch 9/10
5/5 [==============================] - 0s 1ms/step - loss: 1.0828 - s

In [110]:
model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
my_dense_19 (MyDense)        (None, 16)                80        
_________________________________________________________________
tf_op_layer_Tanh_9 (TensorFl [(None, 16)]              0         
_________________________________________________________________
my_dense_20 (MyDense)        (None, 3)                 51        
_________________________________________________________________
tf_op_layer_Softmax_9 (Tenso [(None, 3)]               0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [111]:
model.save('mydense.h5')

#### 解决方案：我们主要看传入__init__接口时有哪些配置参数，然后在get_config内一一的将它们转为字典值并且返回使用，以Mylayer为例

In [64]:
def config_config(self):
    config = super(Linear, self).get_config()
    config.update({'units': self.units})
    return config

# get_config的作用：获取该层的参数配置，以便模型保存时使用

### 注意二：若模型保存(model.save)报错 Unable to create link (name already exists),则可能是自定义层的build中创建初始矩阵时，name属性没写，会导致model.save报错

In [96]:
import tensorflow as tf
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        self.units = units
        super(MyDense, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='random_normal', trainable=True)
        super(MyDense, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})
        return config

In [98]:
inputs = tf.keras.Input(shape=(4,))
x = MyDense(units=16)(inputs)
x = tf.nn.tanh(x)
x = MyDense(units=3)(x)
predictions = tf.nn.softmax(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(data, labels, batch_size=32, epochs=1, shuffle=True)

model.save('mydense.h5')

5/5 [==============================] - 0s 893us/step - loss: 1.0961 - sparse_categorical_accuracy: 0.3333


RuntimeError: Unable to create link (name already exists)

### 注意三：加载时可能会报 Unknown layer 的错误

In [100]:
model = tf.keras.models.load_model("mydense.h5")

ValueError: Unknown layer: MyDense

### 解决方案: 首先，建立一个字典，该字典的键是自定义网络层时设定的名字，其值为该自定义网络层的类名，该字典将用于加载模型时使用。
### 然后，在tf.keras.models.load_model内传入custom_objects告知如何解析重建自定义网络层

In [112]:
_custom_objects = {
    "MyDense": MyDense,
}

In [113]:
new_model = tf.keras.models.load_model("mydense.h5", custom_objects=_custom_objects)

In [114]:
y_pred = new_model.predict(data)
np.argmax(y_pred, axis=1)

array([1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1])

### 注意四：当我们自定义一个网络层其名字与默认的tf.keras网络层一样时，可能会报出一些奇怪的问题，其实是因为重命名了

#### 试一下 tf.serving 自定义层

In [115]:
tf.saved_model.save(new_model, 'my_saved_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_saved_model/assets


In [116]:
restored_saved_model = tf.saved_model.load('my_saved_model')
f = restored_saved_model.signatures["serving_default"]

In [118]:
data[1]

array([5.5, 2.4, 3.8, 1.1])

In [124]:
f(input_11 = tf.constant([data[1]], dtype=tf.float32))

{'tf_op_layer_Softmax_9': <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.30060118, 0.36116308, 0.33823568]], dtype=float32)>}

### 注意五： 我们在实现自定义网络层时，最好统一在初始化时传入可变参数**kwargs,这是因为在model推理时，有时我们需要对所有构成该模型的网络层进行统一传参